# Filtering data and getting stats on a specific month

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import datetime
from collections import Counter
from  itertools import chain

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [2]:
subreddit = 'france'
month = 9
year = 2022

# Importing data

In [3]:
comments = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_comments_merged.parquet', engine='pyarrow')
assert len(comments[comments.duplicated(['comment_id'])]) == 0, "Meh, I found some duplicated comments IDs in the dataframe"

posts = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_posts_merged.parquet', engine='pyarrow')
assert len(posts[posts.duplicated(['post_id'])]) == 0, "Meh, I found some duplicated post IDs in the dataframe"

print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 204240 comments
We have 11286 posts


# Filtering on comments published in the specific month

In [4]:
comments = comments[(comments['year_comment'] == year)
    & (comments['month_comment'] == month)]
posts = posts[(posts['year_post'] == year)
    & (posts['month_post'] == month)]
print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 106720 comments
We have 6927 posts


# Analysis

## Global
### Number of posts
### Number of comments
### Top 3 posts with the highest number of comments (+ links)
### Average number of comments per posts
### Number of unique authors (posts + comments)

In [5]:
nb_posts = posts['post_id'].nunique()
nb_comments = comments['comment_id'].nunique()
top_com1 = posts.nlargest(3,'nb_comment').iloc[:1]
top_com2 = posts.nlargest(3,'nb_comment').iloc[1:2]
top_com3 = posts.nlargest(3,'nb_comment').iloc[2:3]
avg_comments_posts = round(posts['nb_comment'].mean(), 2)
nb_active_users = np.unique(comments['author_post'] + comments['author_comment']).size

## Language
### Top 3 words appearing the most in titles
### Top 3 words appearing the most in comments

In [9]:
top_titles_words = pd.Series(' '.join(posts['title_processed']).split()).value_counts()[:10]
top_titles_words

france    320
pas       212
queen     185
«         181
plus      179
»         170
new       133
|         130
ne        127
best      121
dtype: int64

In [15]:
top_comments_words = pd.Series(' '.join(comments['text_processed']).split()).value_counts()[:10]
top_comments_words

pas      70674
c'est    48364
ne       28210
plus     26680
bien     13778
faire    13639
j'ai     12402
qu'il     8162
non       7793
c’est     6855
dtype: int64

## Flairs
### Number of posts per flair
### Posts with the highest number of comments per flair (linked)

In [41]:
nb_posts_flairs = posts.groupby(['flair']).size().sort_values(ascending=False)[:3].reset_index()
biggest_post_flair1 = posts[posts['flair'] == nb_posts_flairs['flair'][0]].nlargest(1,'nb_comment')
biggest_post_flair2 = posts[posts['flair'] == nb_posts_flairs['flair'][1]].nlargest(1,'nb_comment')
biggest_post_flair3 = posts[posts['flair'] == nb_posts_flairs['flair'][2]].nlargest(1,'nb_comment')